In [ ]:
# 필요 라이브러리 정의
import os

import torchvision.models as models
import torch
import torch.nn as nn
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img 


import collections
from sklearn.cluster import KMeans


def GetDLModel():
  # load the pretreained Resnet model
  # 모델 불러와서 일부 키 값 resnet50 model에 맞게 수정

  device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")


  checkpoint2 = torch.load('/content/drive/MyDrive/resnet50_market_xent.pth.tar', map_location=device, encoding='latin1')
  checkpoint2['state_dict']['fc.weight'] = checkpoint2['state_dict'].pop('classifier.weight')
  checkpoint2['state_dict']['fc.bias'] = checkpoint2['state_dict'].pop('classifier.bias')

  # 기존의 resnet50을 market1501을 학습 시킨 모델과 출력층이 동일하게 구성
  res2 = models.resnet50()
  x = res2.fc.weight
  x = torch.narrow(x, 0, 0, 751)
  y = res2.fc.bias
  y = torch.narrow(y, 0, 0, 751)

  res2.fc.weight = nn.Parameter(x)
  res2.fc.bias = nn.Parameter(y)
  res2.fc.out_features=751

  # 모델 덮어 씌우기
  res2.load_state_dict(checkpoint2['state_dict'])
  res2.to(device) # 가중치를 GPU 계산과 CPU 계산 중 하나로 통일

  return res2

def getImages():
  # 이미지 폴더에서 이미지 셋 리스트에 저장

  # PATH of Cropped and target images
  path = '/content/drive/MyDrive/ZEPETO2'
  os.chdir(path)

  images = []

  # 이미지 파일 리스트에 삽입
  with os.scandir(path) as files:
      for file in files:
          if file.name.endswith('.png'):
              images.append(file.name)
  return images

# Bulid feature extractor class by using pretained resnet
# feature extractor class 설계
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    self.conv1 = model.conv1
    self.bn1 = model.bn1
    self.relu = model.relu
    self.maxpool = model.maxpool
    self.layer1 = model.layer1
    self.layer2 = model.layer2
    self.layer3 = model.layer3
    self.layer4 = model.layer4
    self.global_avgpool = model.avgpool
    self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
    return
  def featuremaps(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    return x
  def forward(self, x):
    f = self.featuremaps(x)
    v = self.global_avgpool(f)
    v = v.view(v.size(0), -1)
    v = self.pool(v)
    v = self.pool(v)
    v = self.pool(v)
    #v = self.pool(v)
    #v = self.pool(v)
    #v = self.pool(v)
    return v
  
# 이미지 특징 추출 함수
def extracting_feature(model, images, target_name, transform):
  device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
  features_ = []
  extract_feature = FeatureExtractor(model)
  for i in range(len(images)):
    if i==len(images):
      break
    path = os.path.join('/content/drive/MyDrive/ZEPETO2', images[i])
    img = load_img(path, target_size=(224,224))
    img = np.array(img)
    img = transform(img)
    img = img.reshape(1, 3, 224, 224)
    img = img.to(device)
    with torch.no_grad():
      feature = extract_feature(img)
    features_.append(feature.cpu().detach().numpy().reshape(-1))
  return features_
  
def clusters(feat, n_clusters=10):
  kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=5, max_iter=300, random_state=0)
  kmeans.fit(feat)
  result = kmeans.labels_
  labels = np.unique(result)
  return result, labels  


# 이미지 transform 구성
def get_transform():
  transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                              
  ])
  return transform

def play(n_cluster=3, image_name=['target.png']):
  target_name = image_name
  MyResnet = GetDLModel()
  zepeto = getImages()
  trans = get_transform()
  features = extracting_feature(MyResnet, zepeto, target_name, trans)
  features = np.array(features)
  (res, labels) = clusters(features, 3)

  groups = {}
  for file, cluster in zip(zepeto,res):
      if cluster not in groups.keys():
          groups[cluster] = []
          groups[cluster].append(file)
      else:
          groups[cluster].append(file)
  count=0
  for i in range(len(labels)):
    flag=0
    for target in target_name:
      if target in groups[i]:
        flag=1
        break
    if flag==0:
      count+=len(groups[i])   
  return count

cnt = play(3,['target.png'])
print('count: ',cnt)
score = 0
if cnt < 20:
  score=20
elif cnt<50:
  score=50
elif cnt<80:
  score=80
elif cnt<100:
  score=100
elif cnt<150:
  score=150
else:
  score=200
print('score: ', score)
#return score

count:  100
score:  150


In [ ]:
  '''
  군집화 결과 검증 코드
  import matplotlib.pyplot as plt
  def view_cluster(groups, cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 20:
        print(f"Clipping cluster size from {len(files)} to 20")
        files = files[:19]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
  for item in labels:
    if item==-1:
      continue
    print(f'image in {item} cluster')
    view_cluster(groups,item)
    plt.show()
    '''